In [25]:
import pandas as pd
import numpy as np

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Data Exploration

In [26]:
# Combine train and test data for more efficient feature engineering.
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
# print(train.shape, test.shape, data.shape)

In [27]:
# Find out columns that have missing values.
# Item_Weight and Outlet_size have missing values.
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [28]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [29]:
# 1559 products being sold.
# 10 unique outlet stores in this dataset.
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
source                           2
dtype: int64

In [30]:
# Filter categorical variables.
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']

# Exclude ID cols and source.
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]

# Print frequency of categories
# for col in categorical_columns:
#     print('\nFrequency of Categories for varible %s'%col)
#     print(data[col].value_counts())

# Data Cleaning

## Impute Missing Values

In [31]:
# Determine the average weight per item.
item_avg_weight = data.groupby('Item_Identifier').Item_Weight.mean()

# Get a boolean variable specifying missing Item_Weight values.
miss_bool = data['Item_Weight'].isnull() 

# Impute data and check missing values before and after imputation to confirm.
# print('Orignal number of missing values for Item_Weight: %d'% sum(miss_bool))
data.Item_Weight.fillna(0, inplace = True)
for index, row in data.iterrows():
    if(row.Item_Weight == 0):
        data.loc[index, 'Item_Weight'] = item_avg_weight[row.Item_Identifier]
# print('Final number of missing values for Item_Weight: %d'% sum(data['Item_Weight'].isnull()))

In [32]:
data.groupby('Outlet_Identifier').Outlet_Size.value_counts(dropna=False)
data.loc[data.Outlet_Identifier.isin(['OUT010','OUT017','OUT045']), 'Outlet_Size'] = 'Small'
data.Outlet_Size.value_counts()

Small     7996
Medium    4655
High      1553
Name: Outlet_Size, dtype: int64

# Feature Engineering

In [33]:
data.min()
# Item_Visibility should not be 0.

Item_Fat_Content                        LF
Item_Identifier                      DRA12
Item_MRP                             31.29
Item_Outlet_Sales                    33.29
Item_Type                     Baking Goods
Item_Visibility                          0
Item_Weight                          4.555
Outlet_Establishment_Year             1985
Outlet_Identifier                   OUT010
Outlet_Location_Type                Tier 1
Outlet_Size                           High
Outlet_Type                  Grocery Store
source                                test
dtype: object

In [34]:
data.loc[data.Item_Visibility == 0, 'Item_Visibility'] = np.nan

# Aggregate by Item_Identifier
IV_mean = data.groupby('Item_Identifier').Item_Visibility.mean()
data.Item_Visibility.fillna(0, inplace=True)

# Replace 0 values
for index, row in data.iterrows():
    if(row.Item_Visibility == 0):
        data.loc[index, 'Item_Visibility'] = IV_mean[row.Item_Identifier]
        
data.Item_Visibility.describe()

count    14204.000000
mean         0.070458
std          0.050086
min          0.003575
25%          0.031381
50%          0.058064
75%          0.098042
max          0.328391
Name: Item_Visibility, dtype: float64

In [35]:
# Create a broad category of Type of Item.

# Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
# Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [36]:
# Determine the years of operation of a store
# Years:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [37]:
data['MRP_Factor'] = pd.cut(data.Item_MRP, [0,70,130,201,400], labels=['Low', 'Medium', 'High', 'Very High'])

In [38]:
# Modify categories of Item_Fat_Content.

# Change categories of low fat:
# print ('Original Categories:')
# print (data['Item_Fat_Content'].value_counts())

# print ('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
# print (data['Item_Fat_Content'].value_counts())

In [39]:
# Mark non-consumables as separate category in low_fat:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
data['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [40]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# New variable for outlet.
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet', 'MRP_Factor']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [41]:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet', 'MRP_Factor'])

In [42]:
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
O

# Exporting Data

In [43]:
pd.options.mode.chained_assignment = None

# Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year',],axis=1,inplace=True)

# Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

# Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

In [44]:
train.head()

,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Identifier,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,...,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9,MRP_Factor_0,MRP_Factor_1,MRP_Factor_2,MRP_Factor_3
0,FDA15,249.8092,3735.1380,0.016047,9.30,OUT049,14,1,0,0,...,0,0,0,0,0,1,0,0,0,1
1,DRC01,48.2692,443.4228,0.019278,5.92,OUT018,4,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,FDN15,141.6180,2097.2700,0.016760,17.50,OUT049,14,1,0,0,...,0,0,0,0,0,1,1,0,0,0
3,FDX07,182.0950,732.3800,0.022930,19.20,OUT010,15,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,NCD19,53.8614,994.7052,0.014670,8.93,OUT013,26,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [46]:
train_features = train.drop('Item_Outlet_Sales', axis=1)
train_outcome = train.Item_Outlet_Sales
test_features = test